## Genome Geospatial Visualizer
Just a simple coordinates plotter on top of `leaflet` Open Street Map API  

In [1]:
# Import modules
import csv
import folium
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from pathlib import Path

In [2]:
# Load the environment variable in topmost directory
env_path = Path('.')/'.env'
load_dotenv(dotenv_path=env_path)

True

In [3]:
# Set the path towards our data directory
data_dir = str(Path.cwd()) + '/data'

In [4]:
# At this point, parsed kv from the .env file is now present as system environment
# variable and they can be conveniently accessed via `os.getenv()`
BASE_URL = os.getenv("BASE_URL")
GOOGLE_API_KEY = os.getenv("GOOGLE_GEOCODING_API")

In [5]:
def normalize_trajectory(trajectory_txt, trajectory_csv):
    """Return the trajectory data in csv format"""
    with open(trajectory_txt, 'r') as in_file:
        stripped = (line.strip() for line in in_file)
        lines = (line.split(";") for line in stripped if line)
        fin_list = []
        for line in lines:
            msisdn = line[0]
            line_new = line[1:]
            for line_child in line_new:
                line_child_list = line_child.split(',')
                line_child_list.insert(0, msisdn)
                fin_list.append(line_child_list)
        with open(trajectory_csv, 'w') as out_file:
            writer = csv.writer(out_file, delimiter=',')
            header = ('msisdn', 'lng', 'lat', 'timestamp', 'cell_density', 'cell_id')
            writer.writerow(header)
            writer.writerows(fin_list)

In [6]:
def extract_lat_long_via_address(GOOGLE_API_KEY, BASE_URL, address='Jakarta'):
    """Return a 2-tuple of latitude and longitude of place of interest"""
    lat, lng = None, None
    api_key = GOOGLE_API_KEY
    base_url = BASE_URL
    endpoint = f"{base_url}?address={address}&key={api_key}"
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        return None, None
    try:
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
    except:
        pass
    return (lat, lng)

In [7]:
def create_map(centre_coord_tuple, zoom_level=10, tiles='openstreetmap'):
    """Build a map visualization layer centered on a coord tuple with a certain zoom level.
       ADD ON : Add switching between layers/tiles"""
    my_map = folium.Map(location=centre_coord_tuple, zoom_start=zoom_level, tiles=tiles)
    # add cartodbpositron tiles...
    folium.TileLayer('CartoDBPositron').add_to(my_map)
    # add a control for switching between layers
    folium.LayerControl().add_to(my_map)
    return my_map

In [8]:
def apply_marker(csv_file, m_map, df_type):
    """Extract longitude-latitude pair from a given csv file and 
    apply them as coordinate marker to our map according to dataframe type"""
    dataframe = pd.read_csv(csv_file)
    if df_type == 'trajectory':
        for row in dataframe.itertuples():
            folium.Marker(location=[row.lat, row.lng], popup=(row.cell_id),
                          icon=folium.Icon(color='red')).add_to(m_map)
    elif df_type == 'staypoint':
        for row in dataframe.itertuples():
            folium.Marker(location=[row.grid_lat, row.grid_lon], popup=row.kelurahan_name,
                          icon=folium.Icon(color='black')).add_to(m_map)
    return m_map

In [9]:
# Normalize_trajectory data
# use v2 : UPCC only event type '1'
pathin_traj = data_dir + '/trajectory_6281219293050_20190215_v2.txt'
pathout_traj = data_dir + '/trajectory_6281219293050_20190215_v2.csv'

normalize_trajectory(pathin_traj, pathout_traj)

In [10]:
# Set staypoint path
pathout_sp = data_dir + '/staypoint_6281219293050_20190215_v3.csv'

In [11]:
# Try the above create_map
coord_jkt = extract_lat_long_via_address(GOOGLE_API_KEY, BASE_URL, address='Jakarta')
map_jkt = create_map(coord_jkt, zoom_level=12)

In [12]:
# Load our trajectory into the map
map_genome = apply_marker(pathout_traj, map_jkt, 'trajectory')

In [13]:
# Load our staypoint into the map
map_genome = apply_marker(pathout_sp, map_jkt, 'staypoint')

In [14]:
# show our map
map_genome

### Using Trajectory 2019-11-01
#### Fixed UPCC records and LACCIMA Reference

In [15]:
# Set trajectory path and normalize
pathin_new_traj = data_dir + '/trajectory_6281219293050_20191101.txt'
pathout_new_traj = data_dir + '/trajectory_6281219293050_20191101.csv'

normalize_trajectory(pathin_new_traj, pathout_new_traj)

In [16]:
# Reload the map
coord_jkt = extract_lat_long_via_address(GOOGLE_API_KEY, BASE_URL, address='Jakarta')
map_jkt = create_map(coord_jkt, zoom_level=12)

In [17]:
# Load our trajectory into the map and show
map_genome = apply_marker(pathout_new_traj, map_jkt, 'trajectory')

map_genome

### Using Trajectory 2019-11-07
#### After Re-Run

In [18]:
# Set trajectory path and normalize
pathin_new_traj = data_dir + '/trajectory_6281219293050_20191107.txt'
pathout_new_traj = data_dir + '/trajectory_6281219293050_20191107.csv'

normalize_trajectory(pathin_new_traj, pathout_new_traj)

In [19]:
# Reload the map
coord_jkt = extract_lat_long_via_address(GOOGLE_API_KEY, BASE_URL, address='Jakarta')
map_jkt = create_map(coord_jkt, zoom_level=12)

In [20]:
# Load our trajectory into the map and show
map_genome = apply_marker(pathout_new_traj, map_jkt, 'trajectory')

map_genome

In [21]:
# Set staypoint path
pathout_sp = data_dir + '/staypoint_6281219293050_20191107.csv'

In [22]:
# Load our staypoint into the map
map_genome = apply_marker(pathout_sp, map_jkt, 'staypoint')

map_genome